# Commit Chronicle Dataset

This notebook investigates the [Commit Chronicle dataset](https://huggingface.co/datasets/JetBrains-Research/commit-chronicle) introduced in the paper ["From Commit Message Generation to History-Aware Commit Message Completion", ASE 2023](https://arxiv.org/abs/2308.07655) - loading, filtering, EDA and preprocessing

In [1]:
from datasets import load_dataset, load_from_disk
import rootutils
import multiprocessing as mp
from functools import partial

In [2]:
ROOT = rootutils.find_root(".", ".project-root")
OUTPUT_DIR = ROOT / "data/playground"

## Loading and Filtering

Note: Filtering logic is implemented in `CommitChroniclePreprocessor`

In [3]:
SPLIT = "validation"  # we select this split as it's small for our EDA, feel free to change to `train` split if u want
LANGUAGES = ["Go"]

filtered = OUTPUT_DIR / "01-filtered"


# we don't directly reference `LANGUAGES` in the function because in python multiprocessing,
# all functions passed as parameters shouldn't reference variables outside of them
def filter_dataset(example, languages):
    return example["language"] in languages


if not filtered.exists():
    (
        load_dataset("JetBrains-Research/commit-chronicle", "default", split=SPLIT)
        .filter(partial(filter_dataset, languages=LANGUAGES), num_proc=mp.cpu_count())
        .save_to_disk(filtered)
    )
dataset = load_from_disk(filtered)

In [4]:
dataset.select(range(10))

Dataset({
    features: ['author', 'date', 'timezone', 'hash', 'message', 'mods', 'language', 'license', 'repo', 'original_message'],
    num_rows: 10
})

## EDA

What kind of EDA can we possibly do?

## Preprocessing

Columns of interest in the dataset are:
1. `mods` - Contains all file changes information - what files are changed, the type of change made (addition, modification), and the exact file changes.
2. `message` - The (processed) git commit message
3. `author` - (Optional) This will be used if we want to group commits by a certain author and use that as input. This is and advanced use case

We are going to tokenize the `mods` and `message` using two different tokenizer, since `mods` contains code and which is quite different from `message` which is mostly natural language. So, one tokenizer for `mods`, another for `message`.

We'll start with `message`. The output from the tokenization of  `message` will be called `msg_input_ids`.

Note: All the preprocessing logic explored here is implemented in `CommitChroniclePreprocessor`.

In [5]:
from src.data.components.tokenization import add_special_tokens
from transformers import AutoTokenizer

# This is the tokenizer used in the Commit Chronicle dataset
# The rationale behind this choice is yet to be investigated? Someone could investigate and report :)
# OR we may have to train our own tokenizer as suggested by our all-knowing ChatGPT (https://chatgpt.com/share/672e3b64-6b84-8009-a6c9-adac73cf647e)
msg_tokenizer_ = AutoTokenizer.from_pretrained("Salesforce/codet5-base")

# add `sep_token` and `pad_token`
# `sep_token` is necessary when we are training on a history of git diffs (which is an advanced usage and not part of our initial experiments)
# `pad_token` is necessary for correct batch construction.
msg_tokenizer_ = add_special_tokens(msg_tokenizer_, None)

Let's try out commit message tokenization on a single example

In [6]:
msg_input_ids_ = msg_tokenizer_(
    dataset[0]["message"], truncation=False, padding=False, add_special_tokens=False
).input_ids

print(dataset[0]["message"])
print(msg_input_ids_)

Add code to retrieve the correct user directory on windows and linux ... and a somewhat okayish directory on macos"
[986, 981, 358, 4614, 326, 3434, 729, 1867, 603, 9965, 471, 19725, 1372, 471, 279, 18016, 11304, 21194, 1468, 1867, 603, 5318, 538, 6]


Next, we'll look at the tokenization of git commit changes, `mods`. But before we do that, let's examine the structure of the data.

In [7]:
dataset[0]["mods"]

[{'change_type': 'ADD',
  'old_path': None,
  'new_path': 'internal/config/config.go',
  'diff': '+package config\n+\n+import "strings"\n+\n+var (\n+ //AppName is the name representing the application.\n+ AppName = "Cordless"\n+ //AppNameLowercase is the representative name, but lowercase.\n+ //It us used for filepaths and such.\n+ AppNameLowercase = strings.ToLower(AppName)\n+)\n+\n+//Config contains all possible configuration for the application.\n+type Config struct {\n+ Token string\n+}\n+\n+var cachedConfigDir string\n+\n+//GetConfigDirectory is the parent directory in the os, that contains the\n+//settings for the application.\n+func GetConfigDirectory() (string, error) {\n+ if cachedConfigDir != "" {\n+ return cachedConfigDir, nil\n+ }\n+\n+ directory, err := getConfigDirectory()\n+ if err != nil {\n+ return "", err\n+ }\n+\n+ cachedConfigDir = directory\n+ return cachedConfigDir, nil\n+}\n'},
 {'change_type': 'ADD',
  'old_path': None,
  'new_path': 'internal/config/os_darwin.g

We'll need to somehow combine all that information into a single string before tokenization.

In [8]:
def preprocess_mods(mods: list[dict[str, str]], line_sep: str) -> str:
    """
    Transforms a list of all files modifications made in a commit into a single string representation.

    Specifically, adds a header to each file diff (https://git-scm.com/docs/git-diff#_generating_patch_text_with_p)
    and concatenates the results.

    Args:
        mods: A list of files modifications made in a commit.
        line_sep: The line separator to separate each file modification.

    Returns:
        A single string representation of all files modifications made in a commit.
    """
    diff = ""

    for mod in mods:
        if mod["change_type"] == "UNKNOWN":
            continue
        elif mod["change_type"] == "ADD":
            file_diff = f"new file {mod['new_path']}"
        elif mod["change_type"] == "DELETE":
            file_diff = f"deleted file {mod['old_path']}"
        elif mod["change_type"] == "RENAME":
            file_diff = (
                f"rename from {mod['old_path']}{line_sep}rename to {mod['new_path']}"
            )
        elif mod["change_type"] == "COPY":
            file_diff = (
                f"copy from {mod['old_path']}{line_sep}copy to {mod['new_path']}"
            )
        else:
            file_diff = f"{mod['new_path']}"
        diff += file_diff + line_sep + mod["diff"]

    return diff


# Let's test it out
print(preprocess_mods(dataset[0]["mods"], line_sep="\n"))

new file internal/config/config.go
+package config
+
+import "strings"
+
+var (
+ //AppName is the name representing the application.
+ AppName = "Cordless"
+ //AppNameLowercase is the representative name, but lowercase.
+ //It us used for filepaths and such.
+ AppNameLowercase = strings.ToLower(AppName)
+)
+
+//Config contains all possible configuration for the application.
+type Config struct {
+ Token string
+}
+
+var cachedConfigDir string
+
+//GetConfigDirectory is the parent directory in the os, that contains the
+//settings for the application.
+func GetConfigDirectory() (string, error) {
+ if cachedConfigDir != "" {
+ return cachedConfigDir, nil
+ }
+
+ directory, err := getConfigDirectory()
+ if err != nil {
+ return "", err
+ }
+
+ cachedConfigDir = directory
+ return cachedConfigDir, nil
+}
new file internal/config/os_darwin.go
+package config
+
+import (
+ "os/user"
+ "path/filepath"
+)
+
+func getConfigDirectory() (string, error) {
+ //TODO Gotta research this
+
+ currentU

Now onto tokenization of the concatenated git diff or `mods`

In [9]:
from copy import deepcopy

# Here, we just duplicate the message tokenizer, but it could be completely different, or maybe I lied :)
diff_tokenizer_ = deepcopy(msg_tokenizer_)
# diff can be very long, we need to set a limit that our model (and computer resources) can handle
DIFF_MAX_LEN = 512

# again, let's test it
git_diff_ = preprocess_mods(dataset[0]["mods"], line_sep="\n")
diff_input_ids_ = diff_tokenizer_(
    git_diff_,
    truncation=True,
    max_length=DIFF_MAX_LEN
    - 2,  # -2 to account for special tokens (BOS and EOS) to be added later, during batch data construction.
    padding=False,
    add_special_tokens=False,
).input_ids
print(diff_input_ids_[:100], len(diff_input_ids_))

[2704, 585, 2713, 19, 1425, 19, 1425, 18, 3240, 203, 15, 5610, 642, 203, 15, 203, 15, 5666, 315, 10219, 6, 203, 15, 203, 15, 1401, 261, 203, 15, 368, 3371, 461, 353, 326, 508, 5123, 326, 2521, 18, 203, 15, 4677, 461, 273, 315, 39, 517, 2656, 6, 203, 15, 368, 3371, 461, 4070, 3593, 353, 326, 23174, 508, 16, 1496, 12400, 18, 203, 15, 368, 7193, 584, 1399, 364, 3608, 87, 471, 4123, 18, 203, 15, 4677, 461, 4070, 3593, 273, 2064, 18, 774, 4070, 12, 3371, 461, 13, 203, 6975, 203, 15, 203, 15, 759, 809, 1914] 510


Let's put everything together to process `mods` and `message` columns for all rows in the dataset.

In [10]:
def process_example(
    example,
    msg_tokenizer,
    diff_tokenizer,
    diff_max_len,
    diff_line_sep,
    preprocess_mods_func,
):
    msg_input_ids = msg_tokenizer(
        example["message"], truncation=False, padding=False, add_special_tokens=False
    ).input_ids

    git_diff = preprocess_mods_func(example["mods"], line_sep=diff_line_sep)
    diff_input_ids = diff_tokenizer(
        git_diff,
        truncation=True,  # we unfortunately have to truncate the git changes
        max_length=diff_max_len
        - 2,  # -2 to account for special tokens (BOS and EOS) to be added later, during batch data construction.
        padding=False,
        add_special_tokens=False,
    ).input_ids

    return {
        "author": example["author"],
        "message": example["message"],
        "msg_input_ids": msg_input_ids,
        "diff_input_ids": diff_input_ids,
        "repo": example["repo"],
        "language": example["language"],
    }


processed = OUTPUT_DIR / "02-processed"
if not processed.exists():
    (
        dataset.map(
            partial(
                process_example,
                msg_tokenizer=msg_tokenizer_,
                diff_tokenizer=diff_tokenizer_,
                diff_max_len=DIFF_MAX_LEN,
                diff_line_sep="\n",
                preprocess_mods_func=preprocess_mods,
            ),
            num_proc=mp.cpu_count(),
        )
        .select_columns(
            ["author", "msg_input_ids", "diff_input_ids", "language", "repo"]
        )
        .save_to_disk(processed)
    )
dataset = load_from_disk(processed)

In [11]:
dataset.select(range(10))

Dataset({
    features: ['author', 'msg_input_ids', 'diff_input_ids', 'language', 'repo'],
    num_rows: 10
})